# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234287892525                   -0.50    8.0         
  2   -7.250158332078       -1.80       -1.40    1.0   7.18ms
  3   -7.250838124829       -3.17       -1.86    1.0   7.99ms
  4   -7.250897939341       -4.22       -1.84    2.0   8.65ms
  5   -7.251334934776       -3.36       -2.77    1.0   7.76ms
  6   -7.251337994932       -5.51       -3.09    2.0   8.77ms
  7   -7.251338738284       -6.13       -3.68    2.0   8.53ms
  8   -7.251338781352       -7.37       -3.94    2.0   8.91ms
  9   -7.251338797330       -7.80       -4.36    2.0   8.98ms
 10   -7.251338798446       -8.95       -4.87    1.0   7.94ms
 11   -7.251338798677       -9.64       -5.37    2.0   9.08ms
 12   -7.251338798695      -10.72       -5.52    3.0   10.5ms
 13   -7.251338798703      -11.11       -6.03    1.0   7.58ms
 14   -7.251338798704      -11.99       -6.57    2.0   8.82ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03822198370405575
[ Info: Arnoldi iteration step 2: normres = 0.4788392730097802
[ Info: Arnoldi iteration step 3: normres = 0.3585253271564609
[ Info: Arnoldi iteration step 4: normres = 1.0098926774338726
[ Info: Arnoldi iteration step 5: normres = 0.34069711513979084
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.91e-01, 5.26e-02, 2.01e-01, 1.89e-01, 2.05e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5364219513777895
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (8.62e-02, 1.51e-01, 4.82e-01, 1.13e-01, 9.26e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07893693999818666
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.92e-03, 3.24e-02, 1.33e-02, 3.17e-02, 5.54e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09934510001434946
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.70e-